# Using CDP Databases
Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

### Connecting to the database

CDP Seattle uses Firebase's 'Cloud Firestore' to store our data. However, a properly setup database host and associated database module _should_ have the same functionality.

Here is how to connect to the Seattle database for **read only** operations.

**Note:** This notebook connects to the staging instance of Seattle's Firestore database. To use production data, connect to `cdp-seattle`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase

db = CloudFirestoreDatabase("stg-cdp-seattle")
db

<CloudFirestoreDatabase [stg-cdp-seattle]>

### Retrieving a single item
If you know the id of an item in a table, please use the `select_row_by_id` function provided.

In [2]:
event = db.select_row_by_id(table="event", id="0e3bd59c-3f07-452c-83cf-e9eebeb73af2")
event

{'event_id': '0e3bd59c-3f07-452c-83cf-e9eebeb73af2',
 'video_uri': 'http://video.seattle.gov:8080/media/council/gen_062717V.mp4',
 'created': datetime.datetime(2019, 4, 21, 23, 58, 4, 832481),
 'event_datetime': '2017-06-27T00:00:00',
 'body_id': '6f38a688-2e96-4e33-841c-883738f9f03d',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2016/2017-gender-equity-safe-communities-and-new-americans-committee?videoid=x78448'}

### Retrieving many items from a table

You may not know the id's of items you are looking for. In that case, use the `select_rows_as_list` function provided.

In [3]:
events = db.select_rows_as_list(table="event")
events[0]

{'event_id': '0e3bd59c-3f07-452c-83cf-e9eebeb73af2',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2016/2017-gender-equity-safe-communities-and-new-americans-committee?videoid=x78448',
 'video_uri': 'http://video.seattle.gov:8080/media/council/gen_062717V.mp4',
 'created': datetime.datetime(2019, 4, 21, 23, 58, 4, 832481),
 'event_datetime': '2017-06-27T00:00:00',
 'body_id': '6f38a688-2e96-4e33-841c-883738f9f03d'}

### Joining with other tables

In the above event results, notice that a `body_id` is returned for each event. To attach body details to this we can use the python package `pandas` and query the `body` table. Let's first put each of the query results into `pandas.DataFrame` objects.

In [4]:
import pandas as pd

In [5]:
events = pd.DataFrame(events)
events.head()

,body_id,created,event_datetime,event_id,source_uri,video_uri
0,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.832481,2017-06-27T00:00:00,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...
1,ebf7278b-debd-4d19-b0fd-ab1bdeb0f595,2019-04-26 05:07:41.088262,2018-08-08T00:00:00,157aabee-5b7d-4821-9b5d-31f96457b67a,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...
2,f92ad6c2-3994-465e-bc8c-9abeb777ebd2,2019-04-26 15:06:34.285370,2012-06-11T00:00:00,1802e1eb-5225-4219-b2a9-c5336259ce76,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/Tr...
3,44a794de-6e1d-43dd-ac9f-317924345bdb,2019-04-21 23:23:30.958242,2017-12-06T00:00:00,1ffb5920-3c23-4084-b287-cef74c9c56c8,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ed...
4,c28e1141-60f2-421d-9c17-e629b57e8890,2019-04-21 23:31:38.209946,2019-04-11T00:00:00,226d8033-666c-49aa-831d-37d04d693106,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/li...


In [6]:
bodies = db.select_rows_as_list("body")
bodies = pd.DataFrame(bodies)
bodies.head()

,body_id,created,description,name
0,02ed9fbe-0b45-4574-bd73-2d6c29fdc73f,2019-04-26 14:54:34.886772,None,Select Committee on Utility Strategic Planning
1,0b1ae9bd-53c3-4882-900c-fed1b59e2442,2019-04-26 08:28:42.919760,None,"Gender Equity, Safe Communities, New Americans..."
2,318d0a2a-93d1-417b-aa26-e37ad61b81e8,2019-04-21 23:24:47.435263,None,Finance and Culture Committee
3,44a794de-6e1d-43dd-ac9f-317924345bdb,2019-04-21 23:23:30.575342,None,"Education, Equity, and Governance Committee"
4,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.378827,None,"Gender Equity, Safe Communities & New Americans"


In [7]:
expanded_event_details = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
expanded_event_details.head()

,body_id,created_event,event_datetime,event_id,source_uri,video_uri,created_body,description,name
0,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.832481,2017-06-27T00:00:00,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...,2019-04-21 23:58:04.378827,None,"Gender Equity, Safe Communities & New Americans"
1,ebf7278b-debd-4d19-b0fd-ab1bdeb0f595,2019-04-26 05:07:41.088262,2018-08-08T00:00:00,157aabee-5b7d-4821-9b5d-31f96457b67a,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-04-26 05:07:40.696253,None,Finance & Neighborhoods Committee
2,f92ad6c2-3994-465e-bc8c-9abeb777ebd2,2019-04-26 15:06:34.285370,2012-06-11T00:00:00,1802e1eb-5225-4219-b2a9-c5336259ce76,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/Tr...,2019-04-26 15:06:34.021277,None,Seattle Transportation Benefit District Board
3,44a794de-6e1d-43dd-ac9f-317924345bdb,2019-04-21 23:23:30.958242,2017-12-06T00:00:00,1ffb5920-3c23-4084-b287-cef74c9c56c8,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ed...,2019-04-21 23:23:30.575342,None,"Education, Equity, and Governance Committee"
4,c28e1141-60f2-421d-9c17-e629b57e8890,2019-04-21 23:31:38.209946,2019-04-11T00:00:00,226d8033-666c-49aa-831d-37d04d693106,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/li...,2019-04-21 23:31:37.572810,None,Select Committee on the Library Levy


`left_on` refers to the column name in the dataframe calling the operation.
In this case, the column to merge on is `body_id` in the events results.

Similarly, `right_on` refers to the column name in the dataframe to be passed to the operation.
In this case, the column to merge on is `id` in the bodies results.

`suffixes` is a tuple to use for adding suffixes to any columns with the same name between the two dataframes.
Commonly for CDP query results, these are columns such as `created`, which provide a `datetime` value for when that row was stored in the database.

Please refer to `pandas.DataFrame.merge` documentation for more details.

[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

### Filtering

You may notice that the function: `select_rows_as_list` allows for additional parameters to be passed: `filters`, `order_by`, and `limit`. Unfortunately, at this time, `filters` is not available for the open access portions of the API. So while you can provide them to the function, they are not actually used. Because of this, you must do filtering on your end. Fortunately however, `pandas` works well for these types of operations.

[stackoverflow](https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas)

In [8]:
gender_eq = "Gender Equity, Safe Communities & New Americans"
gender_eq_events = expanded_event_details.loc[expanded_event_details["name"] == gender_eq]
gender_eq_events

,body_id,created_event,event_datetime,event_id,source_uri,video_uri,created_body,description,name
0,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.832481,2017-06-27T00:00:00,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...,2019-04-21 23:58:04.378827,None,"Gender Equity, Safe Communities & New Americans"
